In [10]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

# read preprocessed data
cwd = os.getcwd()
df = pd.read_csv(cwd + "/data_csv/preprocessing_data.csv")

# replace string emotion label with integer
dictionary = {'emotion':{'HAPPINESS': 0,'NEUTRAL': 1, 'SURPRISE': 2, 'ANGER': 3, 'SADNESS': 4, 'DISGUST': 5}}
df.replace(dictionary, inplace = True)

# train test split
train, test = train_test_split(df, test_size=0.1)


In [11]:
# export train data to train.csv
train = pd.DataFrame(train, columns=["image", "emotion"])
train.to_csv(cwd + "/data_csv/train.csv")

train.groupby('emotion').count()

,image
emotion,
0,2134
1,2004
2,81
3,64
4,32


In [12]:
# export test data to test.csv
test = pd.DataFrame(test, columns=["image", "emotion"])
test.to_csv(cwd + "/data_csv/test.csv")

test.groupby('emotion').count()

,image
emotion,
0,225
1,232
2,5
3,14
4,4


In [13]:
df = train

In [14]:
import cv2, math, numpy as np, dlib
from imutils import face_utils

# emotion list
# emotions = [0, 1, 2, 3, 4, 5] 
# clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(cwd + "/predictor/shape_predictor_68_face_landmarks.dat")

error = []
mlist = []
distlist = []

# run each row in train.csv
for idx, row in df.iterrows():
    imagePath = cwd + "/cleaned_images/" + row.image
    image = cv2.imread(imagePath)

    rects = detector(image, 0)

    if len(rects) == 0:
        error.append(row.image)

    xlist = []
    ylist = []
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks and convert the facial landmark (x, y)
        shape = predictor(image, rect)
        shape = face_utils.shape_to_np(shape)

        # loop over coordinates, draw them on the image and store coordinates in two lists
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
            xlist.append(x)
            ylist.append(y)

    # get mean of both axes to determine centre of gravity
    xmean = np.mean(xlist) 
    ymean = np.mean(ylist)

    # plot central face on image
    cv2.circle(image, (int(xmean), int(ymean)), 1, (0, 255, 0), -1)

    # find distance between mouth
    mavg = np.mean([ylist[61] - ylist[67], ylist[62] - ylist[66], ylist[63] - ylist[65]])
    mlist.append(mavg)

    # find distance between every poin to central point
    templist = []
    for i in range(17, 68):
        dist = math.sqrt(math.pow(xlist[i] - xmean, 2) + math.pow(ylist[i] - ymean, 2))
        templist.append(dist)
    distavg = np.mean(dist)
    distlist.append(distavg)
  
    # show the output image with the face detections + facial landmarks
    cv2.imshow("Output", image)# 
    k = cv2.waitKey(5) & 0xFF
    if k == 68:
        break

print(len(error))

0


In [15]:
# add feature to df
df['mouth_distance'] = mlist
df['average_distance'] = distlist

df.to_csv(cwd + "/data_csv/train.csv")